In [1]:
import pandas as pd
import sqlite3 as lite
con = lite.connect('../data/election.db')
cur = con.cursor() 

Creating the Candidats table

In [114]:
cur.execute("DROP TABLE IF EXISTS Candidats")
cur.execute("""
    CREATE TABLE Candidats(
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Sexe TEXT,
    Prenom TEXT,
    Nom TEXT, 
    UNIQUE (Prenom, Nom))""")


Initiating geo tables

In [31]:
dept2014_xl = pd.ExcelFile('../data/input/Dep_15_Resultats_com_T1_c.xlsx')
dept2014_T1 = dept2014_xl.parse('Feuil1')
geos = dept2014_T1[['Libellé de la commune',
                    'Code de la commune',
                    'Libellé du canton',
                    'Code du canton',
                    'Libellé du département',
                    'Code du département']]


Creating the Communes table from dept. elections

In [12]:
cur.execute("DROP TABLE IF EXISTS Departements")
cur.execute("""
CREATE TABLE Departements(
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom TEXT,
    DeptCode TEXT)""")

In [18]:
dept = geos[['Libellé du département', 'Code du département']].drop_duplicates()
cur.executemany("""
INSERT INTO Departements(Id, Nom, DeptCode) 
VALUES(NULL, ?, ?)""", dept.values.tolist())
con.commit()

In [83]:
cur.execute("DROP TABLE IF EXISTS Communes")
cur.execute("""
CREATE TABLE Communes(
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom TEXT, 
    CommCode INT, 
    DeptId TEXT,
    UNIQUE (CommCode, DeptId))""")
con.commit()

In [84]:
coms = geos[['Libellé de la commune', 'Code de la commune','Code du département']].drop_duplicates()
dpts_tbl = pd.read_sql_query("SELECT Id, DeptCode FROM Departements", con)
coms[['Code du département']] = coms[['Code du département']].astype(str)
coms_u = pd.merge(coms, 
                  dpts_tbl,
                  left_on = ['Code du département'], 
                  right_on = ['DeptCode'],
                  how = 'left')
coms_u = coms_u[['Libellé de la commune','Code de la commune', 'Id']]
print(coms_u.isnull().sum())
coms_u.tail(n = 2)

Libellé de la commune    0
Code de la commune       0
Id                       0
dtype: int64


,Libellé de la commune,Code de la commune,Id
36534,M'Tsangamouji,513,98
36535,Tsingoni,517,98


In [85]:
cur.executemany("""
INSERT INTO Communes(Id, Nom, CommCode, DeptId) 
VALUES(NULL, ?, ?, ?)""", coms_u.values.tolist())
con.commit()

In [109]:
cur.execute("DROP TABLE IF EXISTS Cantons")
cur.execute("""
CREATE TABLE Cantons(
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom TEXT, 
    CantCode INT, 
    CommId INT)""")

In [111]:
cans = geos[['Code du canton',
             'Libellé du canton',
             'Code de la commune',
             'Code du département']].drop_duplicates()
coms_tbl = pd.read_sql_query("""
    SELECT C.Id, C.CommCode, D.DeptCode 
    FROM Communes C 
    LEFT JOIN Departements D ON DeptId = D.Id""", con)
cans['Code du département'] = cans['Code du département'].astype(str)
cans_u = pd.merge(cans, 
                  coms_tbl,
                  left_on = ['Code du département', 'Code de la commune'], 
                  right_on = ['DeptCode', 'CommCode'],
                  how = 'left')
cans_u = cans_u[['Libellé du canton','Code du canton','Id']]
print(cans_u.isnull().sum())
cans_u.head(n = 2)

Libellé du canton    0
Code du canton       0
Id                   0
dtype: int64


,Libellé du canton,Code du canton,Id
0,Ambérieu-en-Bugey,1,1
1,Ambérieu-en-Bugey,1,2


In [113]:
cur.executemany("""
INSERT INTO Cantons(Id, Nom, CantCode, CommId) 
VALUES(NULL, ?, ?, ?)""", cans_u.values.tolist())
con.commit()